출처 : pytorch 튜트리얼

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True, 
    download = True, 
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [7]:
loss_fn = nn.CrossEntropyLoss()

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y) # cross entropy

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")        

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t + 1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------------
loss: 2.305907  [    0/60000]
loss: 2.290292  [ 6400/60000]
loss: 2.273428  [12800/60000]
loss: 2.268818  [19200/60000]
loss: 2.240290  [25600/60000]
loss: 2.231719  [32000/60000]
loss: 2.225411  [38400/60000]
loss: 2.202897  [44800/60000]
loss: 2.194943  [51200/60000]
loss: 2.152687  [57600/60000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 2.155136 

Epoch 2
----------------------------------
loss: 2.166121  [    0/60000]
loss: 2.155087  [ 6400/60000]
loss: 2.104511  [12800/60000]
loss: 2.118958  [19200/60000]
loss: 2.060967  [25600/60000]
loss: 2.022521  [32000/60000]
loss: 2.031100  [38400/60000]
loss: 1.966468  [44800/60000]
loss: 1.962450  [51200/60000]
loss: 1.880255  [57600/60000]
Test Error: 
 Accuracy: 57.4%, Avg loss: 1.888925 

Epoch 3
----------------------------------
loss: 1.923286  [    0/60000]
loss: 1.889713  [ 6400/60000]
loss: 1.780487  [12800/60000]
loss: 1.812943  [19200/60000]
loss: 1.704446  [25600/60000]
loss: 1.672760  [